### Dependencies

In [ ]:
# Visualization Dependencies
from IPython.display import Image, SVG
import seaborn as sns

# Filepaths, Numpy, Tensorflow
import os
import numpy as np
import tensorflow as tf

# Keras
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras import backend as K
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

#ignore warning messages 
import warnings
warnings.filterwarnings('ignore') 

sns.set()

### Import and Shape Data

In [ ]:
# pip install emnist
# Import Dataset(s)
from emnist import list_datasets
list_datasets()

In [ ]:
# Working with 'letters'
# Import 
from emnist import extract_training_samples
images_train, labels_train = extract_training_samples('letters')
from emnist import extract_test_samples
images_test, labels_test = extract_test_samples('letters')

In [ ]:
# Check shape of training and testing data
print(images_train.shape)
print(labels_train.shape)
print(images_test.shape)
print(labels_test.shape)

In [ ]:
# Plot an image
%matplotlib inline
import matplotlib.pyplot as plt
plt.imshow(images_train[0,:,:], cmap=plt.cm.Greys)

In [ ]:
# Flatten Data
dims = images_train.shape[1] * images_train.shape[2]
X_train = images_train.reshape(images_train.shape[0], dims)
X_test = images_test.reshape(images_test.shape[0], dims)
print("Training Shape:", X_train.shape)
print("Testing Shape:", X_test.shape)

In [ ]:
# Rescale to 0 -> 1 by dividing by max pixel value (255)
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

In [ ]:
# One-Hot Encoding

from keras.utils import np_utils # used to convert array of labeled data to one-hot vector
# should be 26 but out of index?
# Effects accuracy as have a class where their will be no results
num_classes = 27
y_train = np_utils.to_categorical(labels_train, num_classes)
y_test = np_utils.to_categorical(labels_test, num_classes)

### Create and Evaluate Model

In [ ]:
# Empty Sequential model
from tensorflow.keras.models import Sequential
model = Sequential()

#Layers

# 1 - number of elements (pixels) in each image
# Dense layer - when every node from previous layer is connected to each node in current layer
model.add(Dense(500, activation='relu'))

# Second Hidden Layer
model.add(Dense(500, activation='relu'))

# Output Layer - number of nodes corresponds to number of y labels
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
# Compile Model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop' , metrics=['accuracy'])

In [ ]:
# Train Model
model.fit(X_train, y_train, batch_size=128, epochs=10, shuffle=True, verbose=2)

In [ ]:
model.summary()

In [ ]:
# Save Model
model.save("emnist_trained.h5")

In [ ]:
# Load Model
from tensorflow.keras.models import load_model
model = load_model("emnist_trained.h5")

In [ ]:
# Evaluate Model
model_loss, model_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

### Predict Model

A: 1, B: 2, C: 3, D: 4, E: 5, F: 6, G: 7, H: 8, I: 9, J: 10, K: 11, L: 12, M: 13, N: 14, O: 15, P: 16, Q: 17, R: 18, S: 19, T: 20, U: 21, V: 22, W: 23, X: 24, Y: 25, Z: 26

In [ ]:
test = np.expand_dims(X_train[8], axis=0)
test.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(X_train)

In [ ]:
plt.imshow(scaler.inverse_transform(test).reshape(28,28), cmap=plt.cm.Greys)

In [ ]:
model.predict(test).round()

In [ ]:
model.predict_classes(test)

In [ ]:
test = np.expand_dims(X_train[22], axis=0)
test.shape

In [ ]:
plt.imshow(scaler.inverse_transform(test).reshape(28,28), cmap=plt.cm.Greys)

In [ ]:
model.predict(test).round()

In [ ]:
model.predict_classes(test)

### Import Custom Image 

The first test is mainly to see if importing works. I got the "B" (and all the testing letter images) from graphemica.com. 
This site contains larger images of font styles. I thought this affected the learning as the EMNIST set is created from actual writing. 
The first "B" is Times New Roman, and the test resulted in not predicting correctly. The second "B" is DejaVu Sans, Book. This "B" is bolder and more similar to the test data than the Times New Roman "B". This model will most likely work better with bolder images based off these observations. 
Note: These imported images are also NOT handwritten thus affecting conclusions about the model

In [ ]:
file = "imageData/images/test2.png"

In [ ]:
# Convert to numpy array
from tensorflow.keras.preprocessing import image
image_size = (28,28)
im = image.load_img(file, target_size=image_size, color_mode="grayscale")

#scale and flatten
from tensorflow.keras.preprocessing.image import img_to_array
image = img_to_array(im)

image /= 255
img = image.flatten().reshape(-1, 28*28)

img = 1 - img

# Plot and Predict
plt.imshow(img.reshape(28,28), cmap=plt.cm.Greys)
model.predict_classes(img)

In [ ]:
file = "imageData/images/test5.png"

from tensorflow.keras.preprocessing import image
image_size = (28,28)
im = image.load_img(file, target_size=image_size, color_mode="grayscale")

from tensorflow.keras.preprocessing.image import img_to_array
image = img_to_array(im)

image /= 255
img = image.flatten().reshape(-1, 28*28)

img = 1 - img
plt.imshow(img.reshape(28,28), cmap=plt.cm.Greys)
model.predict_classes(img)

### Hand-Written Tests

#### Pencil

In [ ]:
file = "imageData/imPencil/pencil23.jpg"

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array

image_size = (28,28)
im = image.load_img(file, target_size=image_size, color_mode="grayscale")


image = img_to_array(im)

image /= 255
img = image.flatten().reshape(-1, 28*28)

img = 1 - img
plt.imshow(img.reshape(28,28), cmap=plt.cm.Greys)

model.predict_classes(img)

Image is tilted. Will loop through all images in file and rotate using the PILLOW library. The glob library is to help loop through multiple elements in a file.

In [ ]:
# For saving flipped image and then testing
import PIL
from PIL import Image
import glob, os

files = glob.glob("imageData/imPencil/*.jpg")

for infile in files:
    image = Image.open(infile)
    im = image.rotate(270)
    im.thumbnail((28,28), Image.ANTIALIAS)
    im.save(infile + '_thumbnail', 'JPEG')

Try test again with flipped file

In [ ]:
file = "imageData/imPencil/pencil23.jpg_thumbnail"

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array

image_size = (28,28)
im = image.load_img(file, target_size=image_size, color_mode="grayscale")


image = img_to_array(im)

image /= 255
img = image.flatten().reshape(-1, 28*28)

img = 1 - img
plt.imshow(img.reshape(28,28), cmap=plt.cm.Greys)

model.predict_classes(img)

#### Pen

In [ ]:
# For saving flipped image and then testing
import PIL
from PIL import Image
import glob, os

files = glob.glob("imageData/imPen/*.jpg")

for infile in files:
    image = Image.open(infile)
    im = image.rotate(270)
    im.thumbnail((28,28), Image.ANTIALIAS)
    im.save(infile + '_thumbnail', 'JPEG')

In [ ]:
file = "imageData/imPen/pen9.jpg_thumbnail"

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array

image_size = (28,28)
im = image.load_img(file, target_size=image_size, color_mode="grayscale")


image = img_to_array(im)

image /= 255
img = image.flatten().reshape(-1, 28*28)

img = 1 - img
plt.imshow(img.reshape(28,28), cmap=plt.cm.Greys)

model.predict_classes(img)

#### Sharpie

In [ ]:
# For saving flipped image and then testing
import PIL
from PIL import Image
import glob, os

files = glob.glob("imageData/imSharpie/*.jpg")

for infile in files:
    image = Image.open(infile)
    im = image.rotate(270)
    im.thumbnail((28,28), Image.ANTIALIAS)
    im.save(infile + '_thumbnail', 'JPEG')

In [ ]:
file = "imageData/imSharpie/sharpie11.jpg_thumbnail"

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array

image_size = (28,28)
im = image.load_img(file, target_size=image_size, color_mode="grayscale")


image = img_to_array(im)

image /= 255
img = image.flatten().reshape(-1, 28*28)

img = 1 - img
plt.imshow(img.reshape(28,28), cmap=plt.cm.Greys)

model.predict_classes(img)

#### Marker

In [ ]:
# For saving flipped image and then testing
import PIL
from PIL import Image
import glob, os

files = glob.glob("imageData/imMarker/*.jpg")

for infile in files:
    image = Image.open(infile)
    im = image.rotate(270)
    im.thumbnail((28,28), Image.ANTIALIAS)
    im.save(infile + '_thumbnail', 'JPEG')

In [ ]:
file = "imageData/imMarker/marker15.jpg_thumbnail"

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array

image_size = (28,28)
im = image.load_img(file, target_size=image_size, color_mode="grayscale")


image = img_to_array(im)

image /= 255
img = image.flatten().reshape(-1, 28*28)

img = 1 - img
plt.imshow(img.reshape(28,28), cmap=plt.cm.Greys)

model.predict_classes(img)

### Using OpenCV to read handwritten letters

#### Pen

In [ ]:
file = 'imageData/newImages/test1.jpg'
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array

image_size = (28,28)
im = image.load_img(file, target_size=image_size, color_mode="grayscale")


image = img_to_array(im)

image /= 255
img = image.flatten().reshape(-1, 28*28)

img = 1 - img
plt.imshow(img.reshape(28,28), cmap=plt.cm.Greys)

model.predict_classes(img)

In [ ]:
import PIL
from PIL import Image
import glob, os

files = glob.glob("imageData/newImages/test1.jpg")

for infile in files:
    image = Image.open(infile)
    im = image.rotate(270)
    im.thumbnail((28,28), Image.ANTIALIAS)
    im.save(infile + '_thumbnail', 'JPEG')

In [ ]:
file = 'imageData/newImages/test1.jpg_thumbnail'
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array

image_size = (28,28)
im = image.load_img(file, target_size=image_size, color_mode="grayscale")


image = img_to_array(im)

image /= 255
img = image.flatten().reshape(-1, 28*28)

img = 1 - img
plt.imshow(img.reshape(28,28), cmap=plt.cm.Greys)

model.predict_classes(img)

In [ ]:
import cv2
image = cv2.imread('imageData/newImages/test2.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
resized = cv2.resize(gray, (28,28))
# cv2.imshow('gray',gray)
# cv2.waitKey(0)
cv2.imwrite('imageData/newImages/testGray2.jpg', resized)

In [ ]:
file = 'imageData/newImages/testGray2.jpg'
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array

image_size = (28,28)
im = image.load_img(file, target_size=image_size, color_mode="grayscale")


image = img_to_array(im)

image /= 255
img = image.flatten().reshape(-1, 28*28)

img = 1 - img
plt.imshow(img.reshape(28,28), cmap=plt.cm.Greys)

model.predict_classes(img)

In [ ]:
import cv2
image = cv2.imread('imageData/newImages/test3.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
resized = cv2.resize(gray, (28,28))
# cv2.imshow('gray',gray)
# cv2.waitKey(0)
cv2.imwrite('imageData/newImages/testGray3.jpg', resized)

In [ ]:
file = 'imageData/newImages/testGray3.jpg'
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array

image_size = (28,28)
im = image.load_img(file, target_size=image_size, color_mode="grayscale")


image = img_to_array(im)

image /= 255
img = image.flatten().reshape(-1, 28*28)

img = 1 - img
plt.imshow(img.reshape(28,28), cmap=plt.cm.Greys)

model.predict_classes(img)

#### Marker

In [ ]:
import cv2
image = cv2.imread('imageData/newImages/test4.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
resized = cv2.resize(gray, (28,28))
# cv2.imshow('gray',gray)
# cv2.waitKey(0)
cv2.imwrite('imageData/newImages/testGray4.jpg', resized)

In [ ]:
file = 'imageData/newImages/testGray4.jpg'
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array

image_size = (28,28)
im = image.load_img(file, target_size=image_size, color_mode="grayscale")


image = img_to_array(im)

image /= 255
img = image.flatten().reshape(-1, 28*28)

img = 1 - img
plt.imshow(img.reshape(28,28), cmap=plt.cm.Greys)

model.predict_classes(img)